In [4]:
import redis
import pickle
import time
import uuid
import pandas as pd
from io import StringIO
from typing import Any, Optional
from datetime import datetime, timedelta

class RemoteSender:
    def __init__(self, host='220.203.1.124', port=6379, password='Liujie2024'):
        self.redis = redis.Redis(
            host=host, port=port, password=password,
            decode_responses=False
        )
        self.task_queue = 'function_calls'
        self.result_queue = 'function_results'
        self._test_connection()
        print(f"✅ 发送端pandas版本：{pd.__version__}")

    def _test_connection(self):
        try:
            self.redis.ping()
            print("✅ 发送端：Redis连接成功")
        except Exception as e:
            print(f"❌ 发送端：连接失败 - {e}")
            raise

    def call_remote_function(self, func_name: str, *args, **kwargs) -> Any:
        task_id = f"task_{uuid.uuid4().hex[:8]}"
        task = {
            'func_name': func_name,
            'args': args,
            'kwargs': kwargs,
            'task_id': task_id
        }
        self.redis.rpush(self.task_queue, pickle.dumps(task))
        print(f"📤 已调用远程函数：{func_name}（任务ID：{task_id}）")
        return self._get_result(task_id)

    def _get_result(self, task_id: str, timeout=300) -> Any:
        start_time = time.time()
        while time.time() - start_time < timeout:
            result_data = self.redis.blpop(self.result_queue, timeout=10)
            if not result_data:
                continue

            _, res_bytes = result_data
            result = pickle.loads(res_bytes)
            if result['task_id'] == task_id:
                if result['status'] == 'success':
                    return result['result']  # 返回CSV字符串
                else:
                    raise Exception(f"远程执行失败：{result['error']}")
            self.redis.rpush(self.result_queue, res_bytes)
        raise TimeoutError("任务超时")

    def save_to_csv(self, csv_str: Optional[str], filename: str) -> bool:
        """将CSV字符串保存为本地CSV文件（替代Parquet）"""
        if not csv_str:
            print("⚠️ 数据为空，不保存")
            return False
        try:
            # 从CSV字符串恢复DataFrame（兼容所有pandas版本）
            df = pd.read_csv(StringIO(csv_str))
            # 保存为CSV文件
            df.to_csv(filename, index=False)
            print(f"✅ 保存成功：{filename}（{len(df)}条记录）")
            return True
        except Exception as e:
            print(f"❌ 保存失败：{e}")
            return False

def generate_date_range(start_date_str: str, end_date_str: str) -> list:
    """生成从开始日期到结束日期的所有日期字符串（YYYYMMDD格式）"""
    dates = []
    try:
        start_date = datetime.strptime(start_date_str, '%Y%m%d')
        end_date = datetime.strptime(end_date_str, '%Y%m%d')
        
        if start_date > end_date:
            raise ValueError("开始日期晚于结束日期")
            
        current_date = start_date
        while current_date <= end_date:
            dates.append(current_date.strftime('%Y%m%d'))
            current_date += timedelta(days=1)
    except Exception as e:
        print(f"日期处理错误：{e}")
    return dates

if __name__ == "__main__":
    sender = RemoteSender()
    # 定义日期范围：从20250516到20250923
    start_date = '20250923'
    end_date = '20250923'
    
    # 生成日期列表
    date_list = generate_date_range(start_date, end_date)
    print(f"=== 共需获取 {len(date_list)} 天的数据 ===")
    
    # 循环调用获取每日数据
    for i, date in enumerate(date_list, 1):
        print(f"\n=== 正在处理 {i}/{len(date_list)}：{date} ===")
        try:
            # 调用远程函数获取当日数据
            csv_data = sender.call_remote_function('fetch_daily_stock_data', date)
            # csv_data = sender.call_remote_function('fetch_daily_auction_data', date)
            # 保存为CSV文件，文件名包含日期
            sender.save_to_csv(csv_data, f'stock_{date}.csv')
            
            # 适当延迟，避免请求过于频繁
            time.sleep(1)
        except Exception as e:
            print(f"❌ {date} 处理失败：{e}")
            # 失败后也延迟一下，避免快速重试导致的问题
            time.sleep(2)
    
    print("\n=== 所有日期处理完成 ===")

✅ 发送端：Redis连接成功
✅ 发送端pandas版本：2.3.2
=== 共需获取 1 天的数据 ===

=== 正在处理 1/1：20250923 ===
📤 已调用远程函数：fetch_daily_stock_data（任务ID：task_71d8e99d）
✅ 保存成功：stock_20250923.csv（5155条记录）

=== 所有日期处理完成 ===
